##  **Data Extraction — IE434 Project: Energy Prices in Illinois**

#### **Objective**
This notebook extracts, cleans, and structures MISO energy market data for downstream modeling and analysis.  
The goal of this stage is to prepare two standardized datasets:
- **Debug dataset:** small enough to test code quickly (runs < 2 min)
- **Working dataset:** complete, clean dataset for modeling (training will later be limited to ≤ 40 min)


#### **Data Sources**
All input CSVs are stored in the 'raw' directory:
- MISO_Actual_Energy_Price.csv
- MISO_Day_Ahead_Energy_Price.csv
- MISO_Forecasted_Cleared_Actual_Load.csv

In [6]:
!pip -q install gdown

import gdown, pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
base_path = "/content/drive/MyDrive/IE434 - Energy Prices in Illinois (Deep Dive 2)/Data Extraction"
raw_path = f"{base_path}/raw"
debug_path = f"{base_path}/debug"
working_path = f"{base_path}/working"
processed_path = f"{base_path}/processed"

In [32]:
# Import MISO_Actual_Energy_Price data

def to_dt(series):
    return pd.to_datetime(series, format="%m/%d/%Y %I:%M:%S %p", errors="coerce")

actual = pd.read_csv(f"{raw_path}/MISO_Actual_Energy_Price.csv", low_memory=False)
actual = actual.rename(columns={"Date": "timestamp", "LMP": "lmp", "HUB": "hub"})
actual = actual[actual["hub"] == "ILLINOIS.HUB"].copy()

# convert to datetime + dtypes
actual["timestamp"] = to_dt(actual["timestamp"])
actual["lmp"] = pd.to_numeric(actual["lmp"], errors="coerce").astype("float32")
actual["hub"] = actual["hub"].astype("category")
actual = actual.dropna(subset=["timestamp"]).sort_values("timestamp")
actual = actual.set_index("timestamp")[["lmp", "hub"]]

# debug and working splits
debug_actual = actual.iloc[:5000]
working_actual = actual.loc["2017-08-23":"2025-01-31"]

debug_actual.to_pickle(f"{debug_path}/actual_lmp_il_debug.pkl")
debug_actual.to_csv(f"{debug_path}/actual_lmp_il_debug.csv")
working_actual.to_pickle(f"{working_path}/actual_lmp_il_working.pkl")
working_actual.to_csv(f"{working_path}/actual_lmp_il_working.csv")

print("Actual LMP ", debug_actual.shape, working_actual.shape)
actual.typed


Actual LMP  (5000, 2) (46922, 2)


,lmp,hub
timestamp,,
2006-04-10 11:00:00,68.040001,ILLINOIS.HUB
2006-04-10 12:00:00,41.250000,ILLINOIS.HUB
2006-04-10 13:00:00,35.400002,ILLINOIS.HUB
2006-04-10 14:00:00,29.750000,ILLINOIS.HUB
2006-04-10 15:00:00,39.049999,ILLINOIS.HUB


In [33]:
# Import MISO_Day_Ahead_Energy_Price data

dayahead = pd.read_csv(f"{raw_path}/MISO_Day_Ahead_Energy_Price.csv", low_memory=False)
dayahead = dayahead.rename(columns={"Date": "Timestamp", "lmp": "lmp", "node": "hub"})
dayahead = dayahead[dayahead["hub"] == "ILLINOIS.HUB"].copy()

# convert to datetime + dtypes
dayahead["Timestamp"] = to_dt(dayahead["Timestamp"])
dayahead["lmp"] = pd.to_numeric(dayahead["lmp"], errors="coerce").astype("float32")
dayahead["hub"] = dayahead["hub"].astype("category")
dayahead = dayahead.dropna(subset=["Timestamp"]).sort_values("Timestamp")
dayahead = dayahead.set_index("Timestamp")[["lmp", "hub"]]

# debug and working splits
debug_dayahead = dayahead.iloc[:5000]
working_dayahead = dayahead.loc["2017-08-23":"2025-12-31"]

debug_dayahead.to_pickle(f"{debug_path}/dayahead_lmp_il_debug.pkl")
debug_dayahead.to_csv(f"{debug_path}/dayahead_lmp_il_debug.csv")
working_dayahead.to_pickle(f"{working_path}/dayahead_lmp_il_working.pkl")
working_dayahead.to_csv(f"{working_path}/dayahead_lmp_il_working.csv")

print("Day-Ahead LMP ", debug_dayahead.shape, working_dayahead.shape)
dayahead.head()

Day-Ahead LMP  (5000, 2) (70896, 2)


,lmp,hub
timestamp,,
2014-01-01 00:00:00,26.629999,ILLINOIS.HUB
2014-01-01 01:00:00,25.870001,ILLINOIS.HUB
2014-01-01 02:00:00,25.410000,ILLINOIS.HUB
2014-01-01 03:00:00,25.209999,ILLINOIS.HUB
2014-01-01 04:00:00,24.620001,ILLINOIS.HUB


In [35]:
# Import MISO_Forecasted_Cleared_Actual_Load data

load = pd.read_csv(f"{raw_path}/MISO_Forecasted_Cleared_Actual_Load.csv", low_memory=False)
load = load.rename(columns={"DATE": "Timestamp"})
load = load[["Timestamp", "ForecastedLoad", "ClearedLoad", "ActualLoad"]]

# convert to datetime + dtypes
load["Timestamp"] = to_dt(load["Timestamp"])
for c in ["ForecastedLoad", "ClearedLoad", "ActualLoad"]:
    load[c] = pd.to_numeric(load[c], errors="coerce").astype("float32")
load = load.dropna(subset=["Timestamp"]).sort_values("Timestamp")
load = load.set_index("Timestamp")

# debug and working splits
debug_load = load.iloc[:5000]
working_load = load.loc["2017-08-23":"2025-12-31"]

debug_load.to_pickle(f"{debug_path}/miso_load_debug.pkl")
debug_load.to_csv(f"{debug_path}/miso_load_debug.csv")
working_load.to_pickle(f"{working_path}/miso_load_working.pkl")
working_load.to_csv(f"{working_path}/miso_load_working.csv")

print("Load ", debug_load.shape, working_load.shape)
load.head()

Load  (5000, 3) (60182, 3)


,ForecastedLoad,ClearedLoad,ActualLoad
timestamp,,,
2017-08-23 01:00:00,71745.0,65693.0,67376.0
2017-08-23 02:00:00,68985.0,63414.0,65109.0
2017-08-23 03:00:00,67338.0,62239.0,64196.0
2017-08-23 04:00:00,66788.0,61849.0,63973.0
2017-08-23 05:00:00,67886.0,63837.0,66204.0
